making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc),

making sentence-level features (number of words, amount of punctuation), 

or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc)


*******

bow_features function currently grabs lemmas and word frequencies


In [235]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
#!python3 -m spacy download en    # No, let's not do this in the future.

[nltk_data] Downloading package gutenberg to /home/jeremy/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [236]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

In [237]:
# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

# Read 20% of Alice, instead of 10%
#alice = text_cleaner(alice[:int(len(alice)/10)])
#persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
alice = text_cleaner(alice[:int(len(alice)/5)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/5)])

In [238]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [239]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [240]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct and
                     not token.is_stop
                     and token.lemma_ in common_words
                 )]

# Capture the amount of punctuation (check the length of this list)
        punctuation = [token.lemma_
                       for token in sentence
                       if token.is_punct]

        # Populate the row with word counts.
        capwords = 0
        for word in words:
            df.loc[i, word] += 1

# Capture words in all caps that are longer than two characters.
# We don't want to capture numbers, or Spacy's own PRON token for pronouns, or numbers--
# Austen is apparently fond of mentioning years, which look the same in all caps ;)
            if ((word == str.upper(word)) &
               (word != '-PRON-') & 
               (len(word) > 2) &
               (not word.isnumeric())):
                capwords += 1
        
        # Add sentence-level word count, punctuation, number of words in all caps
        df.loc[i,'sentence_length'] = len(sentence)
        df.loc[i,'sentence_punctuation'] = len(punctuation)
        df.loc[i,'sentence_capwords'] = capwords
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [241]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [242]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850


,squire,Duck,power,acre,door,pleasing,actual,capture,legged,clever,...,law,indifference,spending,journey,naval,text_sentence,text_source,sentence_length,sentence_punctuation,sentence_capwords
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67.0,10.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63.0,7.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33.0,4.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Oh, dear, !)",Carroll,3.0,1.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll,6.0,2.0,0.0


In [243]:
word_counts.loc[word_counts['sentence_capwords'] > 0]

,squire,Duck,power,acre,door,pleasing,actual,capture,legged,clever,...,law,indifference,spending,journey,naval,text_sentence,text_source,sentence_length,sentence_punctuation,sentence_capwords
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll,97.0,13.0,2.0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(She, took, down, a, jar, from, one, of, the, ...",Carroll,62.0,8.0,2.0
111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(ALICE, 'S, RIGHT, FOOT, ,, ESQ, .)",Carroll,7.0,2.0,4.0
112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(HEARTHRUG, ,, NEAR, THE, FENDER, ,, (, WITH, ...",Carroll,13.0,5.0,5.0
268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(said, Alice, ;, not, that, she, wanted, much,...",Carroll,37.0,5.0,1.0
277,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(At, last, the, Dodo, said, ,, ', EVERYBODY, h...",Carroll,19.0,6.0,1.0
303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(This, was, the, page, at, which, the, favouri...",Austen,19.0,4.0,3.0


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [244]:
word_counts

,squire,Duck,power,acre,door,pleasing,actual,capture,legged,clever,...,law,indifference,spending,journey,naval,text_sentence,text_source,sentence_length,sentence_punctuation,sentence_capwords
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67.0,10.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63.0,7.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33.0,4.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Oh, dear, !)",Carroll,3.0,1.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll,6.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(A, very, few, days, more, ,, and, Captain, We...",Austen,51.0,6.0,0.0
878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(It, had, been, a, great, disappointment, to, ...",Austen,51.0,4.0,0.0
879,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(But, a, week, must, pass, ;, only, a, week, ,...",Austen,41.0,7.0,0.0
880,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Captain, Wentworth, made, a, very, early, ret...",Austen,26.0,2.0,0.0


In [245]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split


Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [246]:
rfc = ensemble.RandomForestClassifier(n_estimators=100)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9924385633270322

Test set score: 0.8810198300283286


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [247]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2',solver='lbfgs') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(529, 2403) (529,)
Training set score: 0.9716446124763705

Test set score: 0.8611898016997167


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [248]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8960302457466919

Test set score: 0.8045325779036827


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [249]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [250]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [251]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [252]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
done


In [272]:
pd.Series(y_train[y_train=='Austen'].index).describe()

count    348.000000
mean     594.149425
std      167.235259
min      302.000000
25%      449.750000
50%      594.500000
75%      739.250000
max      880.000000
dtype: float64

In [253]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

In [254]:
# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.5531914893617021


col_0,Austen,Carroll
row_0,,
Austen,124,24
Carroll,123,58


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# https://spacy.io/usage/spacy-101

### My modifications--mostly to the bow_features function, unless noted:
* Added sentence length
* It seemed like the dataset was becoming unbalanced--way more Austen than Carroll--so we now read the first 20% of Alice In Wonderland.
* Captured amount of punctuation per sentence
* Captured number of words in all caps with a length > 2

In [255]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
train = sgd.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', sgd.score(X_train, y_train))
print('\nTest set score:', sgd.score(X_test, y_test))

(529, 2403) (529,)
Training set score: 0.9073724007561437

Test set score: 0.8923512747875354


In [256]:
print('\nTest set score:', sgd.score(X_Emma_test, y_Emma_test))
sgd_Emma_predicted = sgd.predict(X_Emma_test)
pd.crosstab(y_Emma_test, sgd_Emma_predicted)


Test set score: 0.49848024316109424


col_0,Austen,Carroll
row_0,,
Austen,84,64
Carroll,101,80


I want to try an SGD classifier with hyperparameters that are not the defaults:

In [257]:
sgd2 = SGDClassifier(loss='log',penalty='l2',n_jobs=-1,learning_rate='adaptive',eta0=.15) #class_weight='balanced',
train2 = sgd2.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', sgd2.score(X_train, y_train))
print('\nTest set score:', sgd2.score(X_test, y_test))

(529, 2403) (529,)
Training set score: 0.9489603024574669

Test set score: 0.8810198300283286


In [258]:
print('\nTest set score:', sgd2.score(X_Emma_test, y_Emma_test))
sgd2_Emma_predicted = sgd2.predict(X_Emma_test)
pd.crosstab(y_Emma_test, sgd2_Emma_predicted)


Test set score: 0.5197568389057751


col_0,Austen,Carroll
row_0,,
Austen,108,40
Carroll,118,63


The "tuned" SGD classifier did a little worse with the Persuasion/Alice dataset, but scored slightly better with Emma/Alice. However, it appears to accomplish this by simply classifying more as "Austen", which means a lot of Carroll's text is misclassified.

In the interest of time, I'm moving on...

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [259]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


Aww, let's choose Burgess' The Adventures of Buster Bear!

In [260]:
buster = gutenberg.raw('burgess-busterbrown.txt')

In [261]:
buster_pattern = re.compile(r"^[CILVX]+\s+([A-Z]{2,}\s+)+$", re.MULTILINE)
buster = buster_pattern.sub('',buster)
# Buster Bear is shorter than Austen's or Carroll's works, so we can use half of it.
buster = text_cleaner(buster[:int(len(buster)/2)])

buster_doc = nlp(buster)
buster_sents = [[sent, "Burgess"] for sent in buster_doc.sents]
buster_sentences = pd.DataFrame(buster_sents)

Similar to the exercise above, I'll combine this text with another author's, and train the model to recognize the two writers.

I'll try Persuasion first, because I think it'll be easier to tell these two texts apart than it would be Buster and Alice.

In [274]:
buster_persuasion_sentences = pd.DataFrame(buster_sents + persuasion_sents)
buster_persuasion_bow = bow_features(buster_persuasion_sentences, common_words)
print('Done.')

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850
Processing row 900
Processing row 950
Processing row 1000
Processing row 1050
Processing row 1100
Done.


In [275]:
buster_persuasion_bow.shape

(1120, 2405)

In [278]:
X_Buster_test = buster_persuasion_bow.drop(['text_sentence','text_source'], 1)
y_Buster_test = buster_persuasion_bow['text_source']

In [282]:
# Try the model I tuned first
print('\nTest set score:', sgd2.score(X_Buster_test, y_Buster_test))
sgd2_Buster_predicted = sgd2.predict(X_Buster_test)
pd.crosstab(y_Buster_test, sgd2_Buster_predicted)


Test set score: 0.4901785714285714


col_0,Austen,Carroll
text_source,,
Austen,549,31
Burgess,313,227


In [283]:
# Try the basic SGD Classifier
print('\nTest set score:', sgd.score(X_Buster_test, y_Buster_test))
sgd_Buster_predicted = sgd.predict(X_Buster_test)
pd.crosstab(y_Buster_test, sgd_Buster_predicted)


Test set score: 0.4589285714285714


col_0,Austen,Carroll
text_source,,
Austen,514,66
Burgess,258,282


These results look similar to what I saw above--the model I tuned gives a higher set score, but it gets there by classifying a lot more of the texts as Austen's, including a lot of not-Austen text. Meanwhile, the basic SGD model gets more wrong overall, but correctly identifies more of the text that wasn't written by Austen.

# Some other stuff I tried for Challenge 0

If I just make up a sentence or two patterned after each writer's style, will the model recognize them as being similar to the intended author?

In [ ]:
predict_sent = [[sent,"Austen"] for sent in nlp(text_cleaner("He was ever so handsome.")).sents]
predict_sentence = pd.DataFrame(predict_sent)
predict_bow = bow_features(predict_sentence, common_words)

In [ ]:
predict_bow.drop(['text_sentence','text_source'],axis='columns')

In [ ]:
print(sgd2.predict(predict_bow.drop(['text_sentence','text_source'],axis='columns')))

In [ ]:
predict_sent = [[sent,"Carroll"] for sent in nlp(text_cleaner("Well, that would never do--as Alice saw it, monarchs should be serious indeed, but the Queen of Hearts was SUCH a drama queen.")).sents]
predict_sentence = pd.DataFrame(predict_sent)
predict_bow = bow_features(predict_sentence, common_words)

In [ ]:
predict_bow

In [ ]:
print(sgd2.predict(predict_bow.drop(['text_sentence','text_source'],axis='columns')))